In [2]:
import numpy as np
import polars as pl
import math
from matplotlib import pyplot as plt

via Ben Lambert
> the first and most important choice in Bayesian (or Frequentist) analysis is which probability model to use to describe a given process

This statement is helpful because it makes clear the centrality of the choice of probability model. With coin tossing, we can justify this from the physical parallels. With biologically, political, or social models then we try to choose the model that imposes as few assumptions as possible. This is the concept of maximum entropy, and justifies why the binomial distribution is reasonable for binary choices, and the normal distribution is reasonable for processes that sum. But note the normal distribution carries risk because the tails are too narrow. In other words, it assumes the additive process is a complete description whereas in reality there are other unknown inputs that mean thicker tails are a better choice.

Working forwards in a generative manner, the parameters are fixed and the resultant data behaves as a valid probability distribution. But working backwards and fixing the data, then the resultant parameter distribution is not a valid probability distribution. When employing the function in this direction, we call it a *likelihood function* to make this distinction clear. Discrete likelihood distributions do not sum to `1`, and continuous likelihood distributions do not integrate to `1` but both are driven by the same *probability model*, and when run in reverse (as a generative model) they do (sum/integrate to `1`).

The implication is that the analyst must consider the mechanism underlying the data generating process.

And whilst we typically write 

$$
p(\theta|data) = \frac{p(data|\theta) \times p(\theta)}{p(data)}
$$

we should write ...

$$
p(\theta|data,model) = \frac{p(data|\theta) \times p(\theta) \times p(model)}{p(data,model)}
$$

but whilst we drop the $p(model)$ term when writing, we should always remember that our conclusions depend on the probability model chosen.



### The equivalence relation

$$
\mathcal{L}(\theta|data) = p(data|\theta)
$$

> We call the above the *equivalence relation* since a likelihood of $\theta$ for a particular data sample is equivalent to the probability of that data sample for that value of $\theta$.

Let's make that concrete by considering a simple coin flip, where the coin is biased, and the probabilty of obtaining heads on any throw is one of 6 discrete values $\theta \in \{0.0, 0.2, 0.4, 0.6, 0.8, 1.0\}$. We now flip the coin twice. This will produce the following table.

In [3]:
# write steps and flips with + 1 as a reminder that python is 0-indexed
steps = 5 + 1
flips = 2 
thetas = np.linspace(0.0, 1.0, num=steps)

probs = []
for theta in thetas:
    for heads in range(flips + 1):
        tails = flips - heads
        p = math.comb(flips, heads) * (theta ** heads) * (1.0 - theta) ** tails
        p = np.round(p, 4) # b/c of floating point rounding errors
        probs.append(p)
probs = np.array(probs).reshape(steps, flips + 1)
probs = pl.DataFrame(probs)
probs

column_0,column_1,column_2
f64,f64,f64
1.0,0.0,0.0
0.64,0.32,0.04
0.36,0.48,0.16
0.16,0.48,0.36
0.04,0.32,0.64
0.0,0.0,1.0


In [16]:
new_names = ["0 heads", "1 heads", "2 heads"]
probs = probs.select([pl.col(old).alias(new) for old, new in zip(probs.columns, new_names)])
probs.with_columns(pl.Series(name="thetas", values=thetas))


0 heads,1 heads,2 heads,thetas
f64,f64,f64,f64
1.0,0.0,0.0,0.0
0.64,0.32,0.04,0.2
0.36,0.48,0.16,0.4
0.16,0.48,0.36,0.6
0.04,0.32,0.64,0.8
0.0,0.0,1.0,1.0


We can see that each row is a valid probability distribution that sums to 1.0.

In [17]:
probs.select(pl.all()).sum_horizontal()

sum
f64
1.0
1.0
1.0
1.0
1.0
1.0


But each column is a likelihood function not a probability distribution because we vary the parameter $\theta$, and the sum of the likelihood functions is not equal to 1.0.

In [18]:
# the sum expression is applied by column by default (hence why we used the sum_horizontal function) in the example above
probs.select(pl.all()).sum()

0 heads,1 heads,2 heads
f64,f64,f64
2.2,1.6,2.2


:::{.callout-important}
The take home message is that $p(\theta|data)$ is not a valid probability distribution.
:::

In [56]:

def calculate_likelihood_binom(n, k, probs):
    """
    Returns standardised likelihoods
    
    Parameters
    ----------
    n : int
        number of trials (assumed constant)
    k : list(int)
        list of number of successes in each trial
    probs: np.ndarray
        The range of probabilities being evaluated
    """
    likelihoods = []
    for prob in probs:
        likelihood = np.prod(binom.pmf(k, n, prob))
        likelihoods.append(likelihood)
    res = pl.Series(likelihoods)
    return res / res.sum()


In [58]:

n, k = 10, [6,7,8,9,6,7,8,9] # trials, successes
probs = np.linspace(0, 1, 10+1)

res = calculate_likelihood_binom(n, k, probs)
from pprint import pprint

pprint(res)

shape: (11,)
Series: '' [f64]
[
	0.0
	3.5405e-42
	3.8709e-25
	9.8502e-16
	1.4153e-9
	0.000024
	0.015649
	0.515843
	0.467961
	0.000523
	0.0
]


In [48]:
from scipy.stats import norm
import numpy as np
import polars as pl

In [ ]:
Miele 9917710

In [59]:
def calculate_likelihood_norm(obs, mean_grid, std_grid):
    """
    Perform a grid search across different possible standard deviations and mean values
    At each point calculate the probability density for the observed data by multiplying together the probabilities for each data point
    """
    likelihoods = []
    for std in std_grid:
        for mean in mean_grid:
            likelihood = np.prod(norm.pdf(obs, mean, std))
            likelihoods.append(likelihood)
    res = pl.Series(likelihoods)
    return res / res.sum()

In [76]:
# estimate the normal distribution for height
obs = [183, 165, 182, 140]
mean_grid = np.linspace(100, 200, 10+1)
std_grid = np.linspace(10,50, 5)

res = calculate_likelihood_norm(obs, mean_grid, std_grid)

In [69]:
m, s = np.meshgrid(mean_grid, std_grid)
s

array([[10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
       [20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.],
       [30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30.],
       [40., 40., 40., 40., 40., 40., 40., 40., 40., 40., 40.],
       [50., 50., 50., 50., 50., 50., 50., 50., 50., 50., 50.]])

In [64]:
norm.pdf([183, 165], 150, 10)

array([0.00017226, 0.01295176])

In [81]:
foo = np.array([[1,2,3],[4,5,6]])

In [84]:
foo.ravel()

array([1, 2, 3, 4, 5, 6])